https://haries.tistory.com/7?category=885610

# 1. 카카오맵_주차장

In [29]:
#!pip install folium

In [30]:
import requests
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
import folium
from folium.plugins import MiniMap

import time
import datetime

from collections import OrderedDict

In [31]:
##카카오 API
def whole_region(keyword, start_x,start_y,end_x,end_y):
    #print(start_x,start_y,end_x,end_y)
    page_num = 1
    # 데이터가 담길 리스트
    all_data_list = []
    
    while(1):
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
        params = {'query': keyword,'page': page_num,
                'rect': f'{start_x},{start_y},{end_x},{end_y}'}
        headers = {'Authorization':'KakaoAK 57a426ba3fb221a28bbb05da0282184d'}
        ## 입력예시 -->> headers = {"Authorization": "KakaoAK f64acbasdfasdfasf70e4f52f737760657"}
        resp = requests.get(url, params=params, headers=headers)
    
        search_count = resp.json()['meta']['total_count']
        print('총 개수',search_count)
    
        if search_count > 45:
            print('좌표 4등분')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            ## 4등분 중 왼쪽 아래
            all_data_list.extend(whole_region(keyword, start_x,start_y,dividing_x,dividing_y))
            ## 4등분 중 오른쪽 아래
            all_data_list.extend(whole_region(keyword, dividing_x,start_y,end_x,dividing_y))
            ## 4등분 중 왼쪽 위
            all_data_list.extend(whole_region(keyword, start_x,dividing_y,dividing_x,end_y))
            ## 4등분 중 오른쪽 위
            all_data_list.extend(whole_region(keyword, dividing_x,dividing_y,end_x,end_y))
            return all_data_list

        else:
            if resp.json()['meta']['is_end']:
                all_data_list.extend(resp.json()['documents'])
                return all_data_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('다음페이지')
                page_num += 1
                all_data_list.extend(resp.json()['documents'])

In [32]:
def overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y):
    # 최종 데이터가 담길 리스트
    overlapped_result = []
    
    # 지도를 사각형으로 나누면서 데이터 받아옴
    for i in range(1,num_x+1): ## 1,10
        end_x = start_x + next_x
        initial_start_y = start_y
        for j in range(1,num_y+1): ## 1,6
            end_y = initial_start_y + next_y
            each_result= whole_region(keyword, start_x,initial_start_y,end_x,end_y)
            overlapped_result.extend(each_result)
            initial_start_y = end_y
        start_x = end_x
    return overlapped_result

In [33]:
def make_map(dfs):
    # 지도 생성하기
    m = folium.Map(location=[36.587496, 127.345869], # 기준좌표: 세종어딘가
                    zoom_start=12)

    # 미니맵 추가
    minimap = MiniMap()
    m.add_child(minimap)

    # 마커
    for i in range(len(dfs)):
        folium.Marker([df['Y'][i],df['X'][i]], # 세종 어딘가
                tooltip=dfs['stores'][i]
                ).add_to(m)
    return m
#     for i in range(len(dfs)):
#         folium.Marker([df['Y'][i],df['X'][i]], # 세종 어딘가
#                 tooltip=dfs['stores'][i],
#                 popup=dfs['place_url'][i],
#                 ).add_to(m)
#     return m

In [34]:
# 시작 x 좌표 및 증가값
keyword = '주차장'
start_x = 127.13
start_y = 36.41
next_x = 0.01
next_y = 0.01
num_x = 27
num_y = 33

# 왼쪽 아래 점 : 36.406781, 127.128471
# 오른쪽 위 점 : 36.741648, 127.404764
# 경도 : 사각형 왼쪽 아래 : 127.13
# 위도 : 사각형 왼쪽 아래 : 36.41
# 좌표 이동 : 0.01
# 움직일 경도 : 27
# 움직일 위도 : 33

In [35]:
overlapped_result = overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y)

# 최종 데이터가 담긴 리스트 중복값 제거
results = list(map(dict, OrderedDict.fromkeys(tuple(sorted(d.items())) for d in overlapped_result)))
X = []
Y = []
stores = []
road_address = []
#place_url = []
ID = []
address_name = []#

# start = time.time()## 시간측정

for place in results:
    
    X.append(float(place['x']))
    Y.append(float(place['y']))
    stores.append(place['place_name'])
    road_address.append(place['road_address_name'])
    #place_url.append(place['place_url'])
    ID.append(place['id'])
    address_name.append(place['address_name'])#
    
    ar = np.array([ID,stores, X, Y, road_address,address_name]).T # 
    # ar = np.array([ID,stores, X, Y, road_address,address_name,place_url]).T
    df = pd.DataFrame(ar, columns = ['ID','stores', 'X', 'Y','road_address','address_name']) #
    # df = pd.DataFrame(ar, columns = ['ID','stores', 'X', 'Y','road_address','address_name','place_url'])
print('total_reuslt_number = ',len(df))


# sec = time.time()-start ## 시간 측정
# times = str(datetime.timedelta(seconds=sec)).split(".")
# times = times[0]
# print(times)

총 개수 0
총 개수 0
총 개수 0
총 개수 2
총 개수 1
총 개수 7
총 개수 13
총 개수 1
총 개수 2
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 4
총 개수 10
총 개수 5
총 개수 3
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수

In [36]:
display(df.head())

,ID,stores,X,Y,road_address,address_name
0,20595952,지하주차장,127.135962459313,36.4478058975321,충남 공주시 우금티로 704,충남 공주시 옥룡동 93
1,17570522,공주대학교옥룡캠퍼스 주차장2,127.133007855898,36.4498710923974,,충남 공주시 옥룡동 191-3
2,17574443,공주대학교 옥룡캠퍼스 주차장1,127.133156495195,36.450689184591,,충남 공주시 옥룡동 187-4
3,18984049,신관금강공원 주차장,127.13242295168149,36.46669467022549,,충남 공주시 신관동 553
4,17557065,공주종합버스터미널 주차장,127.134775595519,36.4684114546741,충남 공주시 신관로 74,충남 공주시 신관동 609


In [37]:
make_map(df)

In [38]:
df.to_csv('parking.csv')

## 1-1.전처리

### 1) 데이터 확인

In [39]:
# 결측치 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            450 non-null    object
 1   stores        450 non-null    object
 2   X             450 non-null    object
 3   Y             450 non-null    object
 4   road_address  450 non-null    object
 5   address_name  450 non-null    object
dtypes: object(6)
memory usage: 21.2+ KB


In [40]:
df_1 = df.copy()

df_1['시'] = df_1['address_name'].str.split(" ").str[0]
df_1['행정동'] = df_1['address_name'].str.split(" ").str[1]

sejong_parking = df_1[df_1['시'] == '세종특별자치시'].reset_index(drop=True)

In [41]:
sejong_parking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            260 non-null    object
 1   stores        260 non-null    object
 2   X             260 non-null    object
 3   Y             260 non-null    object
 4   road_address  260 non-null    object
 5   address_name  260 non-null    object
 6   시             260 non-null    object
 7   행정동           260 non-null    object
dtypes: object(8)
memory usage: 16.4+ KB


### 2) 전용주차장, 전기차충전소는 제외하기

In [42]:
ex_index = sejong_parking.query('stores.str.contains("전기차충전소|전용주차장")', engine='python').index
# [ 20,  33,  45,  50,  82,  83, 101, 111, 112, 138, 157, 160, 161,166, 232, 243, 253, 255]

In [43]:
ex_sejong_parking = sejong_parking.drop(index = ex_index).reset_index(drop=True)

### 3) 피벗테이블 구성

In [65]:
cnt_parking = ex_sejong_parking.pivot_table(index='행정동', values='stores', aggfunc='count', margins=True)
cnt_parking.reset_index(inplace=True)

In [66]:
cnt_parking.columns = ['행정동','주차장']
cnt_parking

,행정동,주차장
0,고운동,2
1,금남면,8
2,다정동,6
3,도담동,48
4,보람동,10
5,부강면,5
6,새롬동,37
7,소담동,6
8,아름동,12
9,연기면,9


# 2) 카카오맵_편의점 

In [46]:
keyword = '편의점'

In [47]:
overlapped_result = overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y)

# 최종 데이터가 담긴 리스트 중복값 제거
results = list(map(dict, OrderedDict.fromkeys(tuple(sorted(d.items())) for d in overlapped_result)))
X = []
Y = []
stores = []
road_address = []
#place_url = []
ID = []
address_name = []#

# start = time.time()## 시간측정

for place in results:
    
    X.append(float(place['x']))
    Y.append(float(place['y']))
    stores.append(place['place_name'])
    road_address.append(place['road_address_name'])
    #place_url.append(place['place_url'])
    ID.append(place['id'])
    address_name.append(place['address_name'])#
    
    ar = np.array([ID,stores, X, Y, road_address,address_name]).T # 
    # ar = np.array([ID,stores, X, Y, road_address,address_name,place_url]).T
    df = pd.DataFrame(ar, columns = ['ID','stores', 'X', 'Y','road_address','address_name']) #
    # df = pd.DataFrame(ar, columns = ['ID','stores', 'X', 'Y','road_address','address_name','place_url'])
print('total_reuslt_number = ',len(df))


# sec = time.time()-start ## 시간 측정
# times = str(datetime.timedelta(seconds=sec)).split(".")
# times = times[0]
# print(times)

총 개수 0
총 개수 0
총 개수 0
총 개수 2
총 개수 3
총 개수 8
총 개수 19
다음페이지
총 개수 19
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 13
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 3
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 3
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수

In [48]:
display(df.head())

,ID,stores,X,Y,road_address,address_name
0,17056961,GS25 옥룡주공점,127.13479159977658,36.44929504933204,충남 공주시 우금티로 719,충남 공주시 옥룡동 98-4
1,19201407,CU 공주미도점,127.13534481560751,36.44926648725075,충남 공주시 우금티로 720,충남 공주시 옥룡동 76-5
2,26937138,GS25 공주옥룡점,127.135124417975,36.4559641931343,충남 공주시 무령로 318,충남 공주시 옥룡동 50-3
3,1966822207,CU 공주옥룡로드점,127.13199134246072,36.45560362842569,충남 공주시 무령로 290,충남 공주시 옥룡동 284-6
4,8664932,CU 공주옥룡점,127.133443725425,36.4542682902645,충남 공주시 우금티로 777,충남 공주시 옥룡동 331-31


In [49]:
make_map(df)

In [50]:
df.to_csv('Convenience.csv')

## 1-1. 전처리
### 딱히 할 게 없음

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 0 to 489
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            490 non-null    object
 1   stores        490 non-null    object
 2   X             490 non-null    object
 3   Y             490 non-null    object
 4   road_address  490 non-null    object
 5   address_name  490 non-null    object
dtypes: object(6)
memory usage: 23.1+ KB


In [52]:
df_2 = df.copy()

In [53]:
df_2['시'] = df_2['address_name'].str.split(" ").str[0]
df_2['행정동'] = df_2['address_name'].str.split(" ").str[1]

sejong_conv= df_2[df_2['시'] == '세종특별자치시'].reset_index(drop=True)

In [54]:
sejong_conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            308 non-null    object
 1   stores        308 non-null    object
 2   X             308 non-null    object
 3   Y             308 non-null    object
 4   road_address  308 non-null    object
 5   address_name  308 non-null    object
 6   시             308 non-null    object
 7   행정동           308 non-null    object
dtypes: object(8)
memory usage: 19.4+ KB


In [55]:
cnt_conv = sejong_conv.pivot_table(index='행정동', values='stores', aggfunc='count', margins=True)
cnt_conv.reset_index(inplace=True)

In [56]:
cnt_conv.columns = ['행정동','편의점']
cnt_conv

,행정동,편의점
0,고운동,16
1,금남면,6
2,나성동,19
3,다정동,9
4,대평동,8
5,도담동,14
6,반곡동,10
7,보람동,14
8,부강면,10
9,새롬동,9
